<h1>Workshop n°4 di GGPL di Emanuele Chiossi, mat. 439468</h1>
<h3>My roof builder</h3>


<h2>Descrizione</h2>

In questo homework lo scopo è quello di realizzare due modelli di tetto di una casa, uno più semplice ed uno più complesso.

La prima cosa da fare è prendere delle immagini di riferimento dal web, io ho scelto le seguenti:
<ul>
<li>tetto semplice: per la seconda immagine si ci riferisce al numero 3
![alt text](image.png)
![alt text](image2.jpg)</li>
<li>tetto complesso:
![alt text](image3.gif)</li>
</ul>

Per facilitarmi la costruzione dei modelli ho deciso di realizzare degli schemi di riferimento sul quale basarmi, con relativi vertici e coordinate di ognuno, propositamente fatto in scala con 1 metro corrispondente a 4 quadretti. 

I vertici identificati da numero e a pedice una s (sotto) sono i vertici ad altezza 0, tutti gli altri sono ad altezza 1.

Questo è lo schema del tetto complesso:

![alt text](schema_rif.jpg)

Lo schema del tetto semplice è facilmente ricavabile da quello complesso, basta eliminare la parte del tetto che sporge sulla destra.

Come suggerito dalle specifiche, sono stati definiti e numerati tutti i vertici partendo dal numero 1. Fatto ciò, il tetto è stato scomposto in facce (celle convesse) senza introdurre nuovi vertici e tramite la funzione MKPOL si sono realizzati entrambi i tetti.

La parte complessa del workshop consiste nel ripulire la struttura in output dalle funzioni drawSimple() e drawComplex(), così da poter avere un risultato non "pieno" e pulito, con la struttura portante del tetto e le coperture ben visibili.

Le variabili utilizzate a tale scopo sono le seguenti:
<ul>
<li>vertexs: vettore di vettori, ognuno dei quali contine le coordinate x, y, z</li>
<li>cells: vettore di vettori, ognuno dei quali rappresenta una faccia, ovvero una cella convessa</li>
<li>roof: tetto realizzato con la funzione MKPOL</li>
<li>skeleton1: 1-scheletro della struttura</li>
<li>skeleton2: 2-scheletro della struttura</li>
<li>cells2vertexs: manipolo la lista delle celle andando a sostituire al numero del vertice X le coordinate del vertice X ottentendo la descrizione delle facce con le coordinate dei punti corrispondenti</li>
<li>cells2vertexsClean: contiene gli elementi della lista cells2vertexs senza doppioni</li>
<li>app: porto al valore 0 i numeri in cells2vertexsClean che sono molto prossimi allo 0 (0.00000000000000001,-0.000000000001) e li memorizzo in app
</li>
<li>cellsClean: dalla lista app, ricostruisco la lista delle celle non considerando i doppioni, quindi modifico opportunamente i valori di ogni faccia</li>
<li>cells2vertexsFinal: da cells2vertexsClean, mi interessano solo i vertici "perimetrali", per cui quelli con z!=0 e i vertici con z=0 e nessun altro vertice sopra</li>
<li>structure: rappresenta la copertura del tetto</li>
</ul>

Ulteriori considerazioni sulla procedura di pulizia delle liste di vertici e celle per una visualizzazione migliore del tetto sono contenute nei commenti del codice, che è il seguente:

In [4]:
#import librerie necessarie
from pyplasm import *
from fpformat import *

#funzione che restutuisce l'HPC raffigurante il tetto della casetta #3, il tetto più semplice
def draw_simple():
    vertexs = [[0,0,0],[4,0,0],[2,1.5,1.25],[2,1.5,0],[0,10,0],[4,10,0],[2,8.5,1.25],[2,8.5,0]]
    cells = [[1,2,3,4],[5,6,7,8],[1,3,4,5,7,8],[2,3,4,6,7,8]]
    roof = MKPOL([vertexs,cells,None])
    return roof

#funzione che restutuisce l'HPC raffigurante il tetto della casa, il tetto più complesso
def draw_complex():
    vertexs = [[0,0,0],[4,0,0],[2,1.5,1.25],[2,1.5,0],[0,10,0],[4,10,0],[2,8.5,1.25],[2,8.5,0],[4,7,0],[7,7,0],[7,5,1],[7,3,0],[4,3,0],[2.345,5,1],[2.345,5,0],[7,5,0]]
    cells = [[1,2,3,4],[5,6,7,8],[1,3,4,5,7,8],[2,3,4,6,7,8],[10,11,12,16],[11,12,13,14,15,16],[9,10,11,14,15,16]]
    roof = MKPOL([vertexs,cells,None])
    return roof

#funzione che "ripulisce" la struttura dell'HPC fornito in input
def ggpl_roof_builder(hpc):
    skeleton1 = OFFSET([.1,.1,.1])(SKEL_1(hpc)) #1-scheletro della struttura 
    skeleton2 = UKPOL(SKEL_2(hpc)) #2-scheletro della struttura 
    vertexs = skeleton2[0] #lista vertici
    cells = skeleton2[1] #lista celle
    #manipolo la lista delle celle andando a sostituire al numero del vertice X le coordinate del vertice X
    #ottentendo la descrizione delle facce con le coordinate dei punti corrispondenti: cells2vertexs 
    cells2vertexs = []
    for c in cells:   #ciclo le celle
        list=[]
        for i in c:   #ciclo sugli elementi della singola cella
            list.append(vertexs[i-1])   #salvo in list le coordinate corrispondenti al singolo vertice
        cells2vertexs.append(list)
    #porto al valore 0 i numeri in cells2vertexsClean che sono molto prossimi allo 0 (0.00000000000000001,-0.000000000001)
    #portiamo i valori in: app
    app = []
    for cvc in cells2vertexs:
        list = []
        for c in cvc:
            l = []
            for i in c:
                x = float(fix(i,3))
                l.append(x)
            list.append(l)
        app.append(list)    
    #elimino eventuali doppioni presenti nella lista
    cells2vertexsClean = []
    for cv in app:
        for i in cv:
            if i not in cells2vertexsClean:
                cells2vertexsClean.append(i)   #appendo gli elementi tralasciando i doppioni
            else: 
                i = cells2vertexsClean.index(i)
    #ricostruisco la lista delle celle non considerando i doppioni, quindi modificando opportunamente i valori di ogni faccia       
    cellsClean = []  
    for cv in app:
        list = []
        for i in cv:
            list.append(cells2vertexsClean.index(i)+1)
        cellsClean.append(list)    
    #mi interessano solo i vertici "perimetrali", per cui quelli con z!=0 e i vertici con z=0 
    #e nessun altro vertice sopra: cells2vertexsFinal 
    cells2vertexsFinal = []
    trovato = 0
    for cvc in cells2vertexsClean:
        if(cvc[2]>0): 
            cells2vertexsFinal.append(cvc) 
        else:
            for cvc2 in cells2vertexsClean:
                if cvc2[2]>0:
                    if cvc2[0]==cvc[0] and cvc2[1]==cvc[1]:
                        cells2vertexsFinal.append(cvc2)
                        trovato=1
            if not trovato:
                cells2vertexsFinal.append(cvc)
            trovato = 0
    #costruisco il tetto        
    structure = MKPOL([cells2vertexsFinal,cellsClean,None])
    structure = STRUCT([T(3)(0.1),structure])
    structure = OFFSET([.1,.1,.1])(structure)
    roof = STRUCT([structure,skeleton1])
    VIEW(roof)
    
#chiamata alla funzione principale    
ggpl_roof_builder(draw_complex())    

<h2>Screenshot dell'oggetto HPC ottenuto: tetto semplice</h2>


![text-alt](tsa.png)

![text-alt](tsb.png)

![text-alt](tsb2.png)

<h2>Screenshot dell'oggetto HPC ottenuto: tetto complesso</h2>


![text-alt](tc.png)

![text-alt](tca.png)

![text-alt](tca2.png)

![text-alt](tcb.png)

![text-alt](tcb2.png)